In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Wed Apr 17 08:58:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              25W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/

In [ ]:
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!git clone https://github.com/aub-mind/arabert
!pip install emoji==1.6.1
!pip install sentencepiece==0.1.96
!pip install seqeval
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.9 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (339/339), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169297 sha256=3d82326cf55e0ed799bb7d1c91145fc4d335cde1f145faab53d4789cec09c375
  Stored in directory: /root/.cache/pip/wheels/3d/c9/af/02caa5725634f27f4e2e43852f67fc9069d014038b236a827e
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Unins

In [ ]:
!git clone https://github.com/AhmedAbdel-Aal/WNER_24_sharedtask.git
!cp -r WNER_24_sharedtask/code/*.py .
!rm -rf WNER_24_sharedtask

Cloning into 'WNER_24_sharedtask'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 64 (delta 22), reused 59 (delta 17), pack-reused 0
Receiving objects: 100% (64/64), 26.10 KiB | 3.26 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
import numpy as np
import torch
import random
import copy
import pandas as pd


from arabert.preprocess import ArabertPreprocessor

from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModel, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures

from arabert.preprocess import ArabertPreprocessor
import numpy as np

from transformers import AutoConfig, AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
from utils import *
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm

In [ ]:
train_data = load_json('/content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split70.json')
dev_data = load_json('/content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split10.json')

def normalize_data(items):
    normalized_data = []
    text_lists = []
    label_list = []
    for item in items:
        sentence_id = item['global_sentence_id']
        sentence = []
        labels = []
        for token_info in item['tokens']:
            word = token_info['token']
            tv = token_info['tags'][0]
            label = tv['value']
            sentence.append(word)
            labels.append(label)
        #sentence = ' '.join(sentence)
        #labels = ' '.join(labels)
        text_lists.append(sentence)
        label_list.append(labels)
    return text_lists, label_list


# Normalize the data
text_train, labels_train = normalize_data(train_data)
text_dev, labels_dev = normalize_data(dev_data)

data loaded from path /content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split70.json
data loaded from path /content/drive/MyDrive/anlp24/WojoodNER_fine_Data/split10.json


In [ ]:
all_labels = list(set([label for sublist in labels_train for label in sublist]))
label_map = { v:index for index, v in enumerate(all_labels)}
inv_label_map = {i: label for i, label in enumerate(all_labels)}

In [ ]:
from data import NERDataset

In [ ]:
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'tokenclassification'
tokenizer_name = 'aubmindlab/bert-base-arabertv02'

In [ ]:
train_dataset = NERDataset(
    texts=text_train,
    tags=labels_train,
    label_map=label_map,
    model_name=model_name,
    tokenizer_name=tokenizer_name,
    max_length=512,
    )

dev_dataset = NERDataset(
    texts=text_dev,
    tags=labels_dev,
    label_map=label_map,
    model_name=model_name,
    tokenizer_name=tokenizer_name,
    max_length=512,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(inv_label_map[label_ids[i][j]])
                preds_list[i].append(inv_label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p):
    preds_list, out_label_list = align_predictions(p.predictions,p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Training

In [ ]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 3e-5,
    fp16 = True, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 8, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 1, # use this to scale batch size without needing more memory
    num_train_epochs= 5,
    weight_decay = 0.01,
    warmup_steps = 500,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'f1',
    greater_is_better = True,
    seed = 25
  )
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark = False

set_seed(25)

In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy Score,Precision,Recall,F1
1,0.118300,0.101031,0.968443,0.872041,0.902057,0.886795
2,0.077300,0.099145,0.971710,0.890415,0.916329,0.903186
3,0.043700,0.114023,0.973048,0.907012,0.910312,0.908659
4,0.024000,0.127639,0.971328,0.892268,0.915489,0.903729
5,0.011200,0.136484,0.972840,0.899178,0.918427,0.908701


TrainOutput(global_step=14455, training_loss=0.08525883570050583, metrics={'train_runtime': 2069.0862, 'train_samples_per_second': 55.882, 'train_steps_per_second': 6.986, 'total_flos': 3.02233634496e+16, 'train_loss': 0.08525883570050583, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.13648423552513123,
 'eval_accuracy_score': 0.9728395919856813,
 'eval_precision': 0.8991780821917809,
 'eval_recall': 0.9184273121589478,
 'eval_f1': 0.9087007683256039,
 'eval_runtime': 45.2423,
 'eval_samples_per_second': 73.029,
 'eval_steps_per_second': 9.129,
 'epoch': 5.0}

In [ ]:
trainer.model.config.label2id = label_map
trainer.model.config.id2label = inv_label_map
trainer.save_model("output_dir2")
train_dataset.tokenizer.save_pretrained("output_dir2")

('output_dir2/tokenizer_config.json',
 'output_dir2/special_tokens_map.json',
 'output_dir2/vocab.txt',
 'output_dir2/added_tokens.json',
 'output_dir2/tokenizer.json')

In [ ]:
!cp -r /content/output_dir2 /content/drive/MyDrive/anlp24/nerflat-arabertv02-e3